### This notebook is a test bed for cleaning gw well data from USGS at SB area

In [21]:
import sys
import os
import matplotlib as plt
import climata
import glob
import numpy as np
import itertools
import pandas as pd
import datetime
%matplotlib inline
plt.rcParams['figure.figsize'] = (14.0, 8.0)

In [3]:
os.chdir('/mnt/c/Users/HCD/UW_work/git/spatial_algrithom/gw_interpolated/USGS_gw/all_raw_data')

In [18]:
os.getcwd()

'/mnt/c/Users/HCD/UW_work/git/spatial_algrithom/gw_interpolated/USGS_gw/all_raw_data'

In [19]:
files = [f for f in os.listdir('.') if os.path.isfile(f)]

In [16]:
fname = '*.txt'

### Read all the files after line 85 from USGS files

In [28]:
files = glob.glob(fname)
#print(files)
for name in files:
    name2=os.path.splitext(name)[0]+'_new.txt'
    #print(name2)
    with open(name) as f:
        with open(name2,'a') as f2:
            for line in islice(f, 85, None):
                f2.write(line)
                #pass


### Clean the files with inconsistent data (shifted columns)

In [8]:
def coltype(string):
    if(string.count(':') == 1):
        return 0
    if(string=="PDT" or string=="PST"):
        return 1
    if(string.count('.') == 1):
        return 2

In [9]:
filename='/mnt/c/Users/HCD/UW_work/spatial_algrithom/Data/gw_interpolated/USGS_gw/procesed_data/test/USGS_342519119391301.txt'
#filename="USGS_342455119405603.txt"
base=os.path.basename(filename)
split=os.path.splitext(base)

outfile=((split[0])[3:])[:-4]+'.txt'

output_file=open(outfile,'w')
with open(filename) as input_file:
    foundusgs=0
    for line in input_file:
        cols = line.split()
        newline=[]
        foundval=0
        if(cols[0] != "USGS" and foundusgs == 1):
            break
        if(cols[0] == "USGS"):
            foundusgs=1
        else:
            continue
        #print(line)
        for col in cols:
            ctype=coltype(col)
            if(ctype!=0 and ctype!=1):
                newline.append(col)
            if(ctype==2):
                foundval=1
                break
        if(foundval):
            output_file.write(newline[0]+" "+newline[1]+" "+newline[2]+" "+newline[3]+" "+newline[4]+"\n")
output_file.close()


### Get the list of site_ID from the filename

In [3]:
fname = '*.txt'
files = glob.glob(fname)

In [4]:
name2=[]

In [5]:
for name in files:
    base=os.path.basename(name)
    split=os.path.splitext(base)
    name2.append((split[0])[5:])

In [6]:
name2ok=pd.DataFrame(name2)
name2ok.to_csv('gw_well.txt')

### Resample the daily GW values into average monthly step

In [34]:
fname = '*.txt'
allf = glob.glob(fname)

In [35]:
i=1
for file in allf:
    file1 = pd.read_table(file, delim_whitespace=True) #skiprows=[0]
    id='WTD_'+str(i)
    i+=1
    file1.columns=['agency','Site_ID','type','Date',id]
    file1['Date'] = pd.to_datetime(file1['Date'])
    file1ok= file1.set_index(pd.DatetimeIndex(file1['Date']))
    file1okf=file1ok.resample("M", how='mean')
    base=os.path.basename(file)
    split=os.path.splitext(base)
    file1okf.to_csv(split[0]+'_ok.csv')

/home/yingjung_cd/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  # Remove the CWD from sys.path while we load stuff.


### Moving files from folder to another folder

In [37]:
import glob, os, shutil

source_dir = '/mnt/c/Users/HCD/UW_work/spatial_algrithom/Data/gw_interpolated/USGS_gw/procesed_data/raw_text_ok' #Path where your files are at the moment
dst = '/mnt/c/Users/HCD/UW_work/spatial_algrithom/Data/gw_interpolated/USGS_gw/procesed_data/test' #Path you want to move your files to
files = glob.iglob(os.path.join(source_dir, "*.csv"))
for file in files:
    if os.path.isfile(file):
        shutil.move(file, dst)

### Merge all the files based on one column

In [90]:
path = r'/mnt/c/Users/HCD/UW_work/spatial_algrithom/Data/gw_interpolated/USGS_gw/procesed_data' # use your path
                    # use your path
all_files = glob.glob(os.path.join(path, "*.csv"))
lodf = [pd.read_csv(f) for f in all_files]

df = pd.concat([d.set_index('Date') for d in lodf], axis=1).reset_index()
df.drop('Site_ID', axis=1, inplace=True)
df.to_csv("all_test_ok1.csv")

/home/yingjung_cd/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
